In [1]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━

In [2]:
import os
import sys
import time
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet
from torch.functional import Tensor
from torch.optim import lr_scheduler
from tqdm import tqdm
import fnmatch

In [3]:
# 환경 변수들이 담긴 dictionary
args = {
    "lr": 0.01,
    "gamma": 0.1,
    "wd": 1e-6,
    "ne": 30,
    "nsc": 10,
    "batch_split": 1,
    "batch": 32,
    "alpha": 0.1,
    "model": "efficientnet-b7",
    "checkpoint_save_directory": "./checkpoint",
    "checkpoint_threshold": 90.0,
}


In [4]:
# 시드 설정으로 다음에도 같은 accuracy 결과가 나올 수 있도록 함
torch.manual_seed(0)
np.random.seed(0)

# 현재PC에서 GPU 이용가능 여부 확인해서 변수에 이용가능한 GPU 종류담기
# apple silicon이면 mps를 사용하고, 아니면 cuda를 이용하고, cuda도 안되면 cpu 이용
device = (
    "mps"
    if torch.backends.mps.is_available()
    else ("cuda" if torch.cuda.is_available() else "cpu")
)
print("device : ", device)

# 데이터로더에서 사용할 배치사이즈.
# batch는 입력으로 받은 batch 값때마다 역전파를 진행하겠다는 의미이고,
# batch_split은 입력으로 받은 batch_split 간격까지 gradient accumulation 하겠다는 의미.
# 자세한건 아래 코드에서..
mini_batch_size = args["batch"] // args["batch_split"]


device :  cuda


In [5]:
class Cutout:
    """랜덤한 위치와 크기의 정사각형으로 이미지의 일부분을 검은색으로 변환하여
    학습 시에 모델이 좀 더 보편적으로 이미지를 학습할 수 있게 도와주는 기법(즉 일반화를 뜻함)"""

    def __init__(self, min_side=30, max_side=60, p=0.5):
        self.max_side = max_side  # 정사각형 한변의 최대 길이
        self.min_side = min_side  # 정사각형 한변의 최소 길이
        self.p = p  # cutout을 진행할 확률

    def __call__(self, image):
        # 0~1 사이의 랜덤하게 생성된 값이 self.p 이상이면 cutout 중지
        if torch.rand([1]).item() > self.p:
            return image

        # 정사각형 한변의 길이를 min_side와 max_side 사이에서 랜덤하게 생성
        side = torch.randint(self.min_side, self.max_side + 1, [1]).item()

        # 이미지의 좌측과 위측의 좌표를 랜덤하게 구하고 (0 ~ image.size(1|2) - side),
        # 랜덤으로 구한 정사각형 한변의 길이를 더해서 우측과 아래측의 좌표를 구한다.
        left = torch.randint(0, image.size(1) - side, [1]).item()
        top = torch.randint(0, image.size(2) - side, [1]).item()
        right = left + side
        bottom = top + side

        # 구해진 좌표를 가지고 슬라이싱을 이용해 이미지의 일부분을 검은색(모든채널을 0)으로 만든다.
        image[:, left:right, top:bottom] = 0
        return image

In [6]:
#  각 이미지에 적용할 변형기법을 파이프라인으로 정의
#  이미지는 각 함수를 통과하여 변형됨
transform_train = transforms.Compose(
    [
        # 보간 기법으로 BILINEAR를 사용하여 160x160사이즈로 이미지를 변환시킨다.
        # 보간 기법을 적용하면 이미지 해상도가 낮아져서 성능이 안좋을줄 알았지만,
        # efficientnet 특성상 이미지 크기가 커야 학습 성능이 더 잘나온다.
        transforms.Resize(160),
        # default 값은 0.5이며, 50% 확률로 이미지 좌우를 반전시킨다.
        transforms.RandomHorizontalFlip(),
        # 이미지를 파이토치의 텐서 객체로 변환하여 trainable하게 변환시킨다.
        transforms.ToTensor(),
        # 이미지를 정규화하여 학습시에 더 빠르고 안정적으로 weights를 변화시켜간다.
        # 위에서 텐서 객체로 변환할 때 0-255범위를 0-1범위로 바꾸기 때문에
        # ToTensor 함수 다음에 Normalize 함수를 적용시켜야한다.
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        Cutout(),
    ]
)

# 테스트 데이터에 대한 변형기법 파이프라인을 설정
transform_test = transforms.Compose(
    [
        # 보간 기법으로 BILINEAR를 사용하여 200x200사이즈로 이미지를 변환시킨다.
        transforms.Resize(200),
        transforms.ToTensor(),
        # 학습때와 같이 정규화를 진행하여 같은 환경을 만들어 줌으로 써
        # 더 정답을 잘 맞출 수 있도록 한다.
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)


In [7]:
# CIFAR100 데이터를 trainset과 testset으로 나눠서 data폴더에 다운로드 받는다.
# 1. 폴더에 이미지데이터가 있으면 메모리에 이미지 데이터를 로드한다. 없으면 다운로드 받고 로드한다.
# 2. transform 파이프라인을 붙여서 나중에 DataLoader가 데이터를 로드할 때
# 각 이미지별로 하나씩 transform 파이프라인을 적용한다.
trainset = torchvision.datasets.CIFAR100(
    root="./data", train=True, download=True, transform=transform_train
)
testset = torchvision.datasets.CIFAR100(
    root="./data", train=False, download=True, transform=transform_test
)


100%|██████████| 169001437/169001437 [00:01<00:00, 97149705.58it/s] 


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
# 정답 라벨과 모델이 예측한 값의 오차를 측정하는 손실함수
# 손실함수로 구한 오차를 토대로 나중에 옵티마이저로 weights를 업데이트 한다.
criterion = nn.CrossEntropyLoss()


In [9]:
def mixup_data(x: Tensor, y, alpha=1.0, lam=1.0, count=0):
    """입력으로 받은 x에 해당하는 이미지들을 mixup해서 데이터 증강
    효과를 볼 수 있게하는 함수"""
    # count가 0이 되는 주기마다 alpha값이 존재할 시 mixup을 진행한다.
    # count의 주기는 args.batch_split이 결정하며,
    # 현재 코드에선 args.batch_split이 1이고, 매 번 mixup을 진행한다.
    if count == 0:
        if alpha > 0:
            # 현재 args 값에 alpha는 0.1로 설정되어있어서
            # lam의 값은 0이나 1에 가까운 값이 난수로 생성되게 된다.
            lam = np.random.beta(alpha, alpha)
        else:
            # lambda가 1인 경우는 mixup을 하지 않는다.
            # 현재 이 코드에선 alpha가 존재하기 때문에 매번 mixup한다.
            lam = 1.0

    # 매 미니배치(32)마다 이 함수가 호출되는데, 그때마다
    # 순서가 0-31의 인덱스 이미지와 0-31범위에 해당하긴하지만
    # 순서가 랜덤으로 생성된 이미지 ex: [1,4,6,8,0,2 ...]
    # 를 mixup 하기위해 batch_size(32)를 인자로 랜덤순열을 생성한다.
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)

    # 위에서 난수로 구한 lam값과 기존 배치 0-31 인덱스와
    # 랜덤순열로 구한 인덱스를 가지고 이미지들을 mix한다.
    mixed_x = lam * x + (1 - lam) * x[index, :]

    # y_a는 0-31인덱스의 정답, y_b는 mixup에 사용된 이미지들의 정답
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


In [10]:
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """loss function(criterion)과 모델의 예측값, mixup 라벨 2개를 받아서
    각각 라벨당 오차를 계산하고 더한 뒤 리턴한다.
    lam값이 1일 경우 y_a에 대한 loss값만 리턴한다."""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


In [11]:
def train(epoch, trainloader) -> None:
    """모델 학습을 담당하는 함수"""
    # 현재 Epoch가 몇번째인지 출력한다.
    print("\nEpoch: %d" % epoch)
    # 모델을 학습모드로 설정한다.
    # 학습모드에서는 gradient 계산 및 dropout이 활성화된다.
    net.train()

    # 현재 epoch에서의 가장 높은 정확도를 구한다.
    # checkpoint 저장 시에 활용된다.
    epoch_acc = 0.0

    # 아래 4개의 지역변수들은 progress_bar 출력을 위해 사용된다.
    train_loss = 0
    correct = 0
    total = 0
    count = 0

    # lam 지역변수는 mixup에서 모델의 예측값이 정답인지 확인할 때 쓴다.
    # mixup_data 함수를 거쳐나오면 lam값이 mutable하게 변하기때문에 항상 1.0이지는 않다.
    lam = 1.0

    # 학습 시작 전에 optimizer의 gradient들을 초기화 해준다.
    optimizer.zero_grad()

    # trainloader에서 설정한 미니배치 단위의 묶음으로 이미지와 정답라벨을 순회하며 가져온다.
    # batch_idx는 구간마다 모델 정확도 및 loss 출력을 위해 사용된다.
    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader, desc="Training")):
        # count가 batch_split과 같아지면 loss function에서 누적계산 해놓은 값을 업데이트한다.
        # batch_split과 같아지기위해 count는 매 루프마다 1씩 증가하는 코드가 아래 존재한다.
        if count == args["batch_split"]:
            # gradient를 업데이트 한다.
            optimizer.step()
            # 새 업데이트를 위해 이전 업데이트에 사용된 값들을 초기화한다.
            optimizer.zero_grad()
            # count를 0으로 맞춰주어서 일정 주기마다 이 if문 안으로 들어올 수 있도록 한다.
            # 예를들어 batch_split이 2라면 짝수 주기로 gradient 업데이트를하고,
            # 1이면 매 루프마다 업데이트한다.
            count = 0
        # batch_split 주기마다 gradient 업데이트 하기위해 count를 1씩 증가해준다.
        count += 1
        # GPU를 사용할 수 있다면 tensor 객체를 GPU에 적재하고, 그렇지 않다면 CPU에 적재한다.
        inputs, targets = inputs.to(device), targets.to(device)

        # inputs: mixup된 이미지
        # targets_a: mixup 중 alpha에 해당하는 이미지 정답라벨
        # targets_b: mixup 중 beta에 해당하는 이미지 정답라벨
        # lam: 이미지 섞인 비율 값
        inputs, targets_a, targets_b, lam = mixup_data(
            inputs, targets, args["alpha"], lam, count
        )
        # efficientnet-b7에 mixup 이미지를 넣어서 predict한다.
        outputs = net(inputs)
        # mixup 이미지의 loss를 계산하는 함수를 이용하여 loss를 계산한다.
        loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
        # batch_split이 2 이상이면 gradient accumulation을 진행하니,
        # loss를 일부분씩만 각각 미니배치에서 취해서 누적계산을 진행한다.
        loss = loss / args["batch_split"]
        # 역전파 진행
        loss.backward()

        # 매 루프마다 train_loss를 누적증가 시킨다.
        train_loss += loss.item()
        # 모델이 예측한 확률 중 가장 높은 라벨만 골라서 추출한다.
        # 1차원 벡터에 32개의 값들이 추출된다. (batch_size가 32여서)
        _, predicted = outputs.max(1)
        # 현 배치의 개수만큼 total에 누적합 한다.
        total += targets.size(0)
        # mixup 이미지를 대상으로 했기때문에 정답을 계산하는 방식도 다르다.
        # target a와 b를 모델이 맞췄다면 lam을 활용해서 비율 조정해서 맞았다고 처리한다.
        correct += (
            lam * predicted.eq(targets_a.data).cpu().sum().float()
            + (1 - lam) * predicted.eq(targets_b.data).cpu().sum().float()
        )

        # accuracy : 맞은 비율 / 전체데이터 개수 * 100.0
        acc = 100.0 * correct / total
        epoch_acc = acc.item()

        if batch_idx % 100 == 99:  # Print every 100 mini-batches
            print(f'[Epoch: {epoch}, Batch: {batch_idx+1}] Loss: {train_loss / (batch_idx+1):.3f}, Accuracy: {acc:.2f}%')

    # 현재 에포크에서 모델이 예측한 정확도가 checkpoint threshold 이상이면
    # checkpoint를 저장한다.
    if epoch_acc > args["checkpoint_threshold"]:
        # 체크포인트를 저장할 경로와 파일이름을 만든다.
        rounded_accuracy = round(epoch_acc, 2)
        saving_ckpt_path = Path(args["checkpoint_save_directory"]) / Path(
            f'{args["model"]}_{rounded_accuracy}.pt'
        )

        print(f"Saving model : {saving_ckpt_path}")

        # 모델의 현재 가중치 상태와 정확도를 기록하여
        # 체크포인트 저장시 입력으로 들어간다.
        state = {
            "net": net.state_dict(),
            "acc": epoch_acc,
        }

        # checkpoint 폴더가 존재하지 않으면 만들어준다.
        if not os.path.isdir(args["checkpoint_save_directory"]):
            os.mkdir(args["checkpoint_save_directory"])

        # 지정한 경로에 체크포인트를 실제로 세이브한다.
        torch.save(state, saving_ckpt_path)


In [12]:
def find_files(directory, pattern):
    """주어진 디렉토리에서 패턴에 맞는 파일들을 찾는다."""
    for root, dirs, files in os.walk(directory):
        for filename in fnmatch.filter(files, pattern):
            yield os.path.join(root, filename)


In [13]:
def test(testloader) -> None:
    """현재 환경 변수에 저장되어있는 모델 이름의 체크포인트를
        전부 읽어 들여서 test를 진행한다."""
    # 모델을 평가모드로 전환해서 dropout 및 gredient 변동이 일어나지 않게한다.
    net.eval()

    # checkpoint들이 있는 directory에서 현재 model의 이름이 들어간
    # 모든 checkpoint를 순회하면서 정확도를 전부 측정한다.
    for checkpoint_path in find_files(
        args["checkpoint_save_directory"], f'*{args["model"]}*'
    ):
        print("Loading checkpoint..")
        # 체크포인트를 load한다.
        checkpoint = torch.load(checkpoint_path)
        # load한 체크포인트를 모델에 적용한다.
        net.load_state_dict(checkpoint["net"])

        test_loss = 0
        correct = 0
        total = 0
        print(f"{'-' * 10} 모델 테스트 시작 {'-' * 10}")
        # gradient를 계산하지 않는 상태에서 모델의 정확도를 측정한다.
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(tqdm(testloader)):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

        # 모델 정확도 측정 및 프린트
        acc = 100.0 * correct / total
        print(
            f"File_path : { checkpoint_path }, Test Loss: {test_loss / len(testloader):.3f}, Test Accuracy: {acc:.2f}%"
        )


In [14]:
# 데이터로더를 이용해서 배치사이즈 크기별로 iterate 할 수 있도록 한다.
# 테스트는 shuffle을 하든 안하든 상관없기 때문에 성능상의 이유로 False이고,
# 트레인은 shuffle을 해야 매 에포크 및 미니배치마다 다양한 조합의 이미지들이
# 배치 정규화 및 mixup 되기때문에 하면 일반화 및 모델 성능에 좋다.
testloader = torch.utils.data.DataLoader(
    testset, batch_size=10, shuffle=False, num_workers=1
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=mini_batch_size, shuffle=True, num_workers=1
)


In [15]:
# 모델은 EfficientNet을 이용한다.
net = EfficientNet.from_pretrained(args["model"], num_classes=100)
net = net.to(device)
optimizer = optim.SGD(
    net.parameters(), lr=args["lr"], momentum=0.9, weight_decay=args["wd"]
)
# StepLR을 이용해서 에포크가 진행될수록 lr를 점점 줄이도록 한다.
lr_sc = lr_scheduler.StepLR(optimizer, step_size=args["nsc"], gamma=args["gamma"])


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth
100%|██████████| 254M/254M [00:01<00:00, 161MB/s]


Loaded pretrained weights for efficientnet-b7


In [16]:
# training을 하기 위한 for loop
for epoch in range(0, args["ne"]):
    train(epoch, trainloader)
    # learning rate를 줄인다.
    lr_sc.step()



Epoch: 0


Training:   0%|          | 0/1563 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Training:   6%|▋         | 100/1563 [01:18<19:09,  1.27it/s]

[Epoch: 0, Batch: 100] Loss: 4.346, Accuracy: 11.50%


Training:  13%|█▎        | 200/1563 [02:38<17:54,  1.27it/s]

[Epoch: 0, Batch: 200] Loss: 3.638, Accuracy: 25.48%


...

[Epoch: 0, Batch: 1400] Loss: 1.436, Accuracy: 63.74%


Training:  96%|█████████▌| 1500/1563 [19:52<00:49,  1.26it/s]

[Epoch: 0, Batch: 1500] Loss: 1.391, Accuracy: 64.65%


Training:  98%|█████████▊| 1532/1563 [20:19<00:24,  1.26it/s]


KeyboardInterrupt: 

In [ ]:
test(testloader)
